## Wstęp do uczenia maszynowego 
### Praca domowa 4
#### Jakub Piwko, Malwina Wojewoda
***

### Pakiety

In [9]:
import pandas as pd
import plotly.express as px
import numpy as np
import sklearn 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')
np.random.seed(23)
from sklearn.model_selection import train_test_split

### Wczytanie i analiza danych

Dane opisują Tajwańskie spółki z lat 1999-2009. Przewidujemy zmienną `Bankrupt?`, która wskazuje, czy dana spółka zbankrutuje. 

In [10]:
df = pd.read_csv('data.csv')
y = df['Bankrupt?']
df.drop('Bankrupt?', axis=1, inplace = True)
df.head()

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,0.780284,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,0.781241,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,0.781550,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 95 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 1    ROA(A) before interest and % after tax                   6819 non-null   float64
 2    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 3    Operating Gross Margin                                   6819 non-null   float64
 4    Realized Sales Gross Margin                              6819 non-null   float64
 5    Operating Profit Rate                                    6819 non-null   float64
 6    Pre-tax net Interest Rate                                6819 non-null   float64
 7    After-tax net Interest Rate                              6819 non-null   float64
 8    Non-industry inco

In [12]:
#Wyróżnienie zbioru, na którym będziemy pracować i zbioru do walidacji
X, X_val, y, y_val = train_test_split(df, y, test_size=0.3)
#Wyróżnienie zbioru treningowego i testowego
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Modele i dobór hiperparametrów 

In [83]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from scipy.stats import poisson,expon

In [49]:
def Scores(model, X, y):
    print(f"Model score: {model.score(X, y)}")
    print(f"Precision score: {precision_score(y, model.predict(X), average='binary')}")
    print(f"Gini score: {roc_auc_score(y, model.predict(X))}")

#### Decision Trees

Na początek wytrenujemy model drzewa decyzyjnego z użyciem domyślnych parametrów. 

In [20]:
tree_base = DecisionTreeClassifier()
tree_base.fit(X_train, y_train)
pred = tree_base.predict(X_test)

In [39]:
Scores(tree_base, X_train, y_train)

Model score: 1.0
Precision score: 1.0
Gini score: 1.0


In [22]:
Scores(tree_base, X_test, y_test)

Model score: 0.9594972067039106
Precision score: 0.3888888888888889
Gini score: 0.7163561076604555


Bazowy model drzewa okazał się już wysoki score, także wyoski gini na zbiorze testowym, na treningowym może być przetrenowany.

Użyjemy teraz grid search to znalezienia optymalnych parametrów. Do tuningu wybraliśmy parametry `max_depth`, `splitter`, `criterion` i `ccp_alpha`.

In [79]:
max_depth=[3, 4, 5]
splitter = ["best", "random"]
criterion=["gini","entrophy"]
ccp_alpha=[0, 0.05, 0.1]
param_grid = dict(max_depth=max_depth, splitter=splitter, criterion=criterion, ccp_alpha=ccp_alpha)

In [73]:
tree = DecisionTreeClassifier()
grid = GridSearchCV(estimator=tree, param_grid=param_grid, cv = 5, n_jobs=-1)

grid_result = grid.fit(X_train, y_train)

print(f"Best score: {grid_result.best_score_}")
print(f"Best parameters: {grid_result.best_params_} ")

Best score: 0.9664773591829793
Best parameters: {'ccp_alpha': 0.05, 'criterion': 'gini', 'max_depth': 3, 'splitter': 'best'} 


In [74]:
tree = DecisionTreeClassifier(ccp_alpha = 0.05,
                             criterion = 'gini',
                             max_depth = 3,
                             splitter = 'best')
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.05, max_depth=3)

In [75]:
sklearn.metrics.confusion_matrix(y_test, tree.predict(X_test))

array([[1386,    0],
       [  46,    0]], dtype=int64)

In [76]:
sklearn.metrics.confusion_matrix(y_train, tree.predict(X_train))

array([[3229,    0],
       [ 112,    0]], dtype=int64)

In [77]:
Scores(tree, X_train, y_train)

Model score: 0.9664771026638731
Precision score: 0.0
Gini score: 0.5


In [78]:
Scores(tree, X_test, y_test)

Model score: 0.9678770949720671
Precision score: 0.0
Gini score: 0.5


Udało się podbić wynik accuracy na zbiorze testowym nieznacznie, ale zapłaciliśmy za to cenę w postaci niższego gini. Precision score spadł (co jest durne, widać na macierzy konfuzji że nie). Model nie jest już przetrenowany. Chociaż po macierzy konfuzji można zobaczyć, że model przypoisuje wszystkim rekordom 0. 

Teraz użyjemy Random Search

In [87]:
tree = DecisionTreeClassifier()
random = RandomizedSearchCV(estimator=tree, param_distributions=param_grid, cv = 5, n_jobs=-1, random_state = 100)

random_result = random.fit(X_train, y_train)

print(f"Best score: {random_result.best_score_}")
print(f"Best parameters: {random_result.best_params_}")

Best score: 0.9664773591829793
Best parameters: {'splitter': 'best', 'max_depth': 3, 'criterion': 'gini', 'ccp_alpha': 0.05}


Z użyciem random search uzyskaliśmy dokładnie taki sam wynik.

#### Random Forest

In [89]:
forest_base = RandomForestClassifier()
forest_base.fit(X_train, y_train)
pred = forest_base.predict(X_test)

In [90]:
Scores(forest_base, X_train, y_train)

Model score: 1.0
Precision score: 1.0
Gini score: 1.0


In [91]:
Scores(forest_base, X_test, y_test)

Model score: 0.9699720670391061
Precision score: 0.8
Gini score: 0.5431175105088148


Podobnie jak w przypadku drzew decyzyjnych, acuuruacy score dla zbioru testowego jest bardzo wysoki, ale wydaje się, że model jest przetrenowany. Małe gini na zbiorze testowym to potwierdza.

In [93]:
max_depth=[3, 4, 5]
n_estimators = [80, 90, 100, 110, 120]
criterion=["gini","entrophy"]
min_samples_split = [2, 5, 10, 20, 50]
param_grid = dict(max_depth=max_depth, n_estimators = n_estimators, criterion=criterion, min_samples_split=min_samples_split)

In [94]:
forest = RandomForestClassifier()
grid = GridSearchCV(estimator=forest, param_grid=param_grid, cv = 5, n_jobs=-1)

grid_result = grid.fit(X_train, y_train)

print(f"Best score: {grid_result.best_score_}")
print(f"Best parameters: {grid_result.best_params_} ")

Best score: 0.9679739176355808
Best parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 120} 


In [121]:
forest = RandomForestClassifier(min_samples_split = 2,
                             criterion = 'gini',
                             max_depth = 5,
                             n_estimators = 120)
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=120)

In [122]:
Scores(forest, X_train, y_train)

Model score: 0.9778509428314875
Precision score: 1.0
Gini score: 0.6696428571428572


In [123]:
Scores(forest, X_test, y_test)

Model score: 0.9692737430167597
Precision score: 0.6666666666666666
Gini score: 0.5427567601480645


Model nie jest już tak przetrenowany po dobraniu hipermarametrów. 

In [118]:
min_samples_split = [i for i in range(2, 30, 3)]
n_estimators = [i for i in range(70, 130, 5)]
param_grid = dict(max_depth=max_depth, n_estimators = n_estimators, criterion=criterion, min_samples_split=min_samples_split)

In [120]:
forest = RandomForestClassifier()
random = RandomizedSearchCV(estimator=forest, param_distributions=param_grid, cv = 5, n_jobs=-1, random_state = 100)

random_result = random.fit(X_train, y_train)

print(f"Best score: {random_result.best_score_}")
print(f"Best parameters: {random_result.best_params_}")

Best score: 0.9661770629145296
Best parameters: {'n_estimators': 85, 'min_samples_split': 20, 'max_depth': 4, 'criterion': 'gini'}


In [124]:
forest = RandomForestClassifier(min_samples_split = 20,
                             criterion = 'gini',
                             max_depth = 4,
                             n_estimators = 85)
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, min_samples_split=20, n_estimators=85)

In [125]:
Scores(forest, X_train, y_train)

Model score: 0.9727626459143969
Precision score: 1.0
Gini score: 0.59375


In [126]:
Scores(forest, X_test, y_test)

Model score: 0.9706703910614525
Precision score: 1.0
Gini score: 0.5434782608695652
